In [ ]:
import math
import numpy as np
import pandas as pd
validator = "Twinstake_processed.csv"
path = f"./{validator}"
df_data= pd.read_csv(path)

In [ ]:
df_data.reset_index(inplace=True)

In [ ]:
print(df_data)

In [ ]:
df_output = pd.DataFrame()
df_output['block Number'] = df_data.block_num
df_output['Atom Staked'] = df_data.atom_staked_v
df_output['Total_Rewards'] = ( df_data.commission_amt/(df_data.v_commission/100)) * 1E-6
df_output['Epoch Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Realized APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in APY'] = df_output.apply(lambda _: '', axis=1)
df_output['% Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)

print(df_output.head())

In [ ]:
# Constants from Network


# Parameters to be varied by validator
for i in range(1,len(df_data)):

  Goal_Bonded                 = float(df_data.Goal_Bonded[i]) 
  Inflation_Rate_Change       = float(df_data.Inflation_Rate_Change[i]) 
  Min_Inflation_Rate          = float(df_data.Min_Inflation_Rate[i]) 
  Max_Inflation_Rate          = float(df_data.Max_Inflation_Rate[i]) 
  Block_length                = float(df_data.block_len[i]) 
  E_Block_length              = float(df_data.Block_length_target[i])
  Blocks_per_year             = float(df_data.Blocks_per_year[i]) 
  minProposerBonus            = float(df_data.min_proposer_bonus[i]) 
  maxProposerBonus            = float(df_data.max_proposer_bonus[i]) 
  Community_Tax               = float(df_data.community_tax[i]) 
    
  Min_Signatures              = float(df_data.Min_Signatures[i]) 
  Total_Fees_Per_Block        = float(df_data.total_block_fees[i]) * 1E-6
  Bonded_Ratio                = float(df_data.percent_staked[i]) /100
  stake                       = float(df_data.atom_staked_v[i]) 
  signatures_ratio            = float(df_data.sign_ratio[i]) 
  E_Inflation_Rate_BH         = float(df_data.inflation_rate[i]) 
  totalSupply                 = float(df_data.total_supply[i]) * 1E-6

  # E_current_Inflation_rate_change = (1 - Bonded_Ratio/Goal_Bonded) * Inflation_Rate_Change
  # E_Inflation_Rate_BH             = Prev_Inflation_Rate + E_current_Inflation_rate_change / Blocks_per_year

    # Check the new inflation rate is between min and max inflation rate
  if E_Inflation_Rate_BH > Max_Inflation_Rate:
    E_Inflation_Rate_BH = Max_Inflation_Rate
  elif E_Inflation_Rate_BH < Min_Inflation_Rate:
    E_Inflation_Rate_BH = Min_Inflation_Rate

  # Real_Inflation_Rate = E_Inflation_Rate_BH * Block_length/ E_Block_length

  E_validator         = stake / (totalSupply*Bonded_Ratio) * ((1+E_Inflation_Rate_BH)**(1/Blocks_per_year)-1) * totalSupply    # technically can cancel totalSupply but left for ease of understanding
  #E_validator         = stake / (totalSupply*Bonded_Ratio) * E_Inflation_Rate_BH/Blocks_per_year * totalSupply
  E_Proposer_Bonus                     = minProposerBonus + (signatures_ratio-Min_Signatures) / (1-Min_Signatures) * (maxProposerBonus-minProposerBonus)      
  E_Proposer_stake                     = totalSupply*Bonded_Ratio/175   # THIS IS POOR ESTIMATE AS VALIDATORS WITH MORE STAKE ARE SELECTED MORE FREQUENTLY, WOULD NEED ALL VALIDATOR'S DATA TO IMPROVE MODEL
  E_transaction_reward_per_atom_staked = Total_Fees_Per_Block * (1 - Community_Tax) / (totalSupply*Bonded_Ratio + E_Proposer_stake*E_Proposer_Bonus)

  E_ProposerTxFees                     = stake * (1 + E_Proposer_Bonus) * E_transaction_reward_per_atom_staked
  E_AttesterTxFees                     = stake * E_transaction_reward_per_atom_staked

  E_Attester = E_validator + E_AttesterTxFees
  E_Proposer = E_validator + E_ProposerTxFees

  P_Proposer = stake/totalSupply    # think this is true even with round robin selection
  P_attester = 1 - P_Proposer


  E_rewards = E_Proposer*P_Proposer + E_Attester*P_attester
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
  df_output['Expected Rewards'][i] = E_rewards
  df_output['Absolute Difference in Rewards'][i] = df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]
  df_output['% Difference in Rewards'][i] = (df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]) / df_output['Epoch Rewards'][i]*100

  APY                                            = E_rewards/stake*Blocks_per_year*E_Block_length/Block_length
  Realized_APY                                   = df_output['Epoch Rewards'][i]/stake*Blocks_per_year*E_Block_length/Block_length
  df_output['Expected APY'][i]                   = APY*100
  df_output['Realized APY'][i]                   = Realized_APY*100     
  df_output['Absolute Difference in Rewards'][i] = E_rewards - df_output['Epoch Rewards'][i]
  df_output['Absolute Difference in APY'][i]     = (df_output['Expected APY'][i]  -   df_output['Realized APY'][i])

print(df_output)

In [ ]:
# df_output.to_csv(f"drive/Shared drives/Model/Data Extraction/COSMOS/single-validators/Output Files/COSMOSValidationOutput-{validator}")

In [ ]:
total_rewards = df_output['Epoch Rewards'][1:].sum()
expected_total = df_output['Expected Rewards'][1:].sum()

In [ ]:
(total_rewards-expected_total)/total_rewards*100